In [ ]:
!pip install --q git+https://github.com/m-bain/whisperx.git         

In [ ]:
import whisperx
import gc

In [ ]:
device = "cpu" # change to "cuda" if your machine has nvidia
batch_size = 4 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

In [ ]:
audio_file = "sam_altman_lex_podcast_367_short.wav"

In [ ]:
audio = whisperx.load_audio(audio_file)

In [ ]:
model = whisperx.load_model("large-v2", device, compute_type=compute_type)


In [ ]:
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)


In [ ]:
result

In [ ]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token="HUGGINGFACE_TOKEN",
                                             device=device)

In [ ]:
diarize_segments = diarize_model(audio, min_speakers=2, max_speakers=2)

In [ ]:
diarize_segments

In [ ]:
diarize_segments.speaker.unique()

In [ ]:
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

In [ ]:
result